@dzhang203
2022-04

[Book Ref: Chapter 19](https://matheusfacure.github.io/python-causality-handbook/19-Evaluating-Causal-Models.html)

# Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from toolz import curry

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.filterwarnings("ignore")

/home/daz/hello_world/tell_me_why/env/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Intro

## Evaluating Causal Models
The goal of a simple causal model is to understand the effect of some treatment $t$ on some outcome $y$, conditional on covariates $x$, i.e. $\frac{\partial y}{\partial t}(t|x)$.

How do we know that a causal model is effectively capturing that true derivative? In academic settings and learning exercises (like this one!), we can generate synthetic data from some DGP where we _know_ the real $\frac{\partial y}{\partial t}(t|x)$--by generating multiple counterfactual outcomes for the same $x$--and see how well our estimated $\hat{\frac{\partial y}{\partial t}}(t|x)$ predicts the true treatment effects.

However, in reality, we won't have different $y$'s for the same $x$ at different intervention levels $t$.

## One Weird Trick: Aggregate on $x$
If $x$ is atomic, then you're never going to have different $y$'s for the same $x$ at different intervention levels $t$. But if you condition on $x$ that includes multiple observation units, then you'll actually be able to observe $\frac{\partial y}{\partial t}(t|x)$!

Note that this is only going to be possible if the variation in $t|x$ is actually exogenous. (Note to self: if it's not, maybe estimate a model to orthogonalize it? Or, manually extract out the random (wiggle) component?)

# Fitting Models

We load data from [github:matheusfacure/python-causality-handbook/data](https://github.com/matheusfacure/python-causality-handbook/tree/master/causal-inference-for-the-brave-and-true/data):

In [3]:
prices = pd.read_csv('../data/ice_cream_sales.csv')  # loads non-exogenous data
prices_exog = pd.read_csv('../data/ice_cream_sales_rnd.csv')  # loads exogenous (DGP'd) data
print(prices.shape)
print(prices_exog.shape)
prices_exog.head()


(10000, 5)
(5000, 5)


,temp,weekday,cost,price,sales
0,25.8,1,0.3,7,230
1,22.7,3,0.5,4,190
2,33.7,7,1.0,5,237
3,23.0,4,0.5,5,193
4,24.4,1,1.0,3,252


The first model we'll train will be a linear regression with interactions so that elasticity can vary between units:

$$ sales_i = \beta_0 + \beta_1 price_i + \beta_2 X_i + \beta_3 X_i price_i + \epsilon_i $$

After fitting this model, we'll be able to make elasticity predictions:

$$ \hat{ \frac{\partial sales}{\partial price}}(price_i, X_i) = \hat{\beta_2} + \hat{\beta_3} X_i $$

Refer to [statsmodels formulas examples](https://www.statsmodels.org/stable/example_formulas.html) if needed; below, the \* operator also includes the individual columns that were multiplied together. 

In [4]:
m1 = smf.ols("sales ~ price*temp + price*C(weekday) + price*temp", data=prices).fit()

# Misc Thoughts

Looking ahead in Ch. 19, Matheus suggests plotting cumulative elasticity curves. What this means is, we're going to collect the estimated elasticities by groups $x$, sort them into high-to-low groups, and then visually see how well-ordered they are. Additionally, we're going to look at how heterogeneous the treatment effects really are, which should give us insight into how much headroom we can actually get from personalization.

However--not covered in Matheus's material--the fundamental model evaluation should still come from looking error in the predicted elasticities themselves. And in this case, we should try to get a test sample with exogenous variation and group them (and potentially orthogonalize the treatment variable) in order to obtain elasticity estimates.